<a href="https://colab.research.google.com/github/rodrigorpereira/deep_learning/blob/master/MNIST_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Auto Encoder applied to MNIST

## Imports and dependency install

In [0]:
!mkdir tensorboard_logs
!mkdir data
!ls

data  sample_data  tensorboard_logs


In [0]:
!pip install pytorch-ignite
!pip install tensorboardX

     |████████████████████████████████| 81kB 24.9MB/s 
     |████████████████████████████████| 225kB 33.6MB/s 


In [0]:
from __future__ import print_function

from torch.utils.data import DataLoader
from torch import nn, cuda, no_grad, randn
from torch.nn import functional as F
from torch.optim import SGD

from torchvision.transforms.transforms import Compose, Resize, ToTensor, Normalize
from torchvision import models
from torchvision import datasets

from ignite.engine import create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.engine.engine import Engine, State, Events
from ignite.utils import convert_tensor
from ignite.contrib.handlers import ProgressBar

from skimage import io, transform
from sklearn.model_selection import train_test_split

import os

try:
    from tensorboardX import SummaryWriter
except ImportError:
    raise RuntimeError('No tensorboardX package is found.')

## Data Loader and TensorBoard Summary

In [0]:
def get_data_loaders(train_batch_size, val_batch_size):
    data_transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])
    
    train_data = datasets.MNIST(
            download=True, root="./data", transform=data_transform, train=True)
    train_loader = DataLoader(
        train_data, batch_size=train_batch_size, shuffle=True, num_workers=4)
    
    val_data = datasets.MNIST(
            download=False, root="./data", transform=data_transform, train=False)
    val_loader = DataLoader(
        val_data, batch_size=val_batch_size, shuffle=False)
    
    return train_loader, val_loader


def create_summary_writer(model, data_loader, log_dir):
    writer = SummaryWriter(logdir=log_dir)
    data_loader_iter = iter(data_loader)
    x, y = next(data_loader_iter)
    try:
        writer.add_graph(model, x)
    except Exception as e:
        print("Failed to save model graph: {}".format(e))
    return writer

## Support and Network Classes

In [0]:
class Interpolate(nn.Module):

    def __init__(self, size, mode):
        super(Interpolate, self).__init__()
        self.interp = F.interpolate
        self.size = size
        self.mode = mode
        
    def forward(self, x):
        x = self.interp(x, size=self.size, mode=self.mode)
        return x

    
class Encoder(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, padding=1, 
                    pool=True):
        super(Encoder, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.pool = pool
        self.encode = nn.Sequential(
                nn.Conv2d(
                    in_channels, out_channels, kernel_size, padding=padding
                ),
                nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        y = self.encode(x)
        result = self.maxpool(y) if self.pool else y
             
        return result


class Decoder(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, size=None,
                 upsampling=False):
        super(Decoder, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.decode = None
        self.size = size

        if upsampling:
            self.decode = nn.Sequential(
                Interpolate(size=self.size, mode='bilinear'),
                nn.Conv2d(in_channels, out_channels, kernel_size, padding=1),
                nn.ReLU())
        else:
            self.decode = nn.Sequential(
                nn.ConvTranspose2d(
                    in_channels, out_channels, kernel_size=2, stride=2
                ),
                nn.ReLU())

    def forward(self, x):
        result = self.decode(x)
        
        return result

## Networks definition

In [0]:
class StackedCAE(nn.Module):

    def __init__(self):
        super(StackedCAE, self).__init__()
        self.conv1 = Encoder(1, 100, 5)
        self.conv2 = Encoder(100, 150, 5)
        self.center = Encoder(150, 200, 3, pool=False)
        self.deconv2 = Decoder(200, 150, 5)
        self.deconv1 = Decoder(150, 100, 5)
        self.output = nn.Conv2d(100, 1, 1)
                
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.center(x)
        x = self.deconv2(x)
        x = self.deconv1(x)
        out = self.output(x)
        
        return out


class VGGAE(nn.Module):

    def __init__(self):
        super(VGGAE, self).__init__()
        self.encoder = list(models.vgg16(pretrained=True).children())[0]
        for param in list(self.encoder.parameters())[:22]:
            param.requires_grad = False

        self.decoder = nn.Sequential (
            nn.Conv2d(512, 512, 3, 1, 1),
            Decoder(512, 256, 3),
            Decoder(256, 128, 3),
            Decoder(128, 64, 3),
            Decoder(64, 3, 3)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


class VAE(nn.Module):

    def __init__(self):
        super(VAE, self).__init__()
        self.n_hidden = 2304
        self.n_z = 8

        self.encoder = nn.Sequential (
            Encoder(1, 32, 4),
            Encoder(32, 64, 4)
        )

        self.fc1 = nn.Linear(2304, self.n_z)
        self.fc2 = nn.Linear(2304, self.n_z)
        self.fc3 = nn.Linear(self.n_z, self.n_hidden)        

        self.decoder = nn.Sequential (
            Decoder(64, 32, 4),3
            nn.Conv2d(32, 1, 1)
        )   
                
    def forward(self, x):
        x = self.encoder(x)
        x = x.view(x.size(0), -1)
        #x = x.view(-1)
        mu, logvar = self.fc1(x), self.fc2(x)

        std = logvar.mul(0.5).exp_()
        esp = randn(*mu.size())
        z = mu + std * esp      

        z = self.fc3(z)
        z = z.view(32, 64, 6, 6)
        out = self.decoder(z)
        return out, mu, logvar

## Train function and training call

In [0]:
def loss_fn(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    # BCE = F.mse_loss(recon_x, x, size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD, BCE, KLD

epochs = 2

train_loader, val_loader = get_data_loaders(32, 1)
model = VAE()

for epoch in range(epochs):
    for idx, (images, _) in enumerate(train_loader):
        out, mu, logvar = model(images)
        print(r'out: {}'.format(out.shape))    
        loss, bce, kld = loss_fn(out, images, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        to_print = "Epoch[{}/{}] Loss: {:.3f} {:.3f} {:.3f}".format(epoch+1, 
                                epochs, loss.data[0]/bs, bce.data[0]/bs, kld.data[0]/bs)
        print(to_print)

# notify to android when finished training
notify(to_print, priority=1)

torch.save(vae.state_dict(), 'vae.torch')

out: torch.Size([32, 1, 12, 12])


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Using a target size (torch.Size([32, 1, 28, 28])) that is different to the input size (torch.Size([32, 1, 12, 12])) is deprecated. Please ensure they have the same size.
  


ValueError: ignored

In [0]:
def update(engine, batch):
    x, _ = batch
    y_pred = model(inputs)
    loss = loss_fn(y_pred, x)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return {'loss': loss.item(),
            'y_pred': y_pred,
            'y': x}


def _prepare_batch(batch, device=None, non_blocking=False):
    
    x, y = batch
    return (convert_tensor(x, device=device, non_blocking=non_blocking),
            convert_tensor(y, device=device, non_blocking=non_blocking))



def _autoencoder_trainer(model, optimizer, loss_fn, metrics={}, device=None):

    def _update(engine, batch):
        model.train()
        optimizer.zero_grad()
        x, _ = _prepare_batch(batch, device=device)
        y_pred = model(x)
        loss = loss_fn(y_pred, x)
        loss.backward()
        optimizer.step()
        return loss.item(), y_pred, x

    def _metrics_transform(output):
        return output[1], output[2]

    engine = Engine(_update)

    for name, metric in metrics.items():
        metric._output_transform = _metrics_transform
        metric.attach(engine, name)

    return engine


def autoencoder_trainer(
        model, optimizer, loss_fn,
        device=None, non_blocking=False,
        prepare_batch=_prepare_batch, 
        output_transform=lambda x, y, y_pred, loss: loss.item()):

    if device:
        model.to(device)

    def _update(engine, batch):
        model.train()
        optimizer.zero_grad()
        x, _ = prepare_batch(batch, device=device, non_blocking=non_blocking)
        y_pred = model(x)
        loss = loss_fn(y_pred, x)
        loss.backward()
        optimizer.step()
        return output_transform(x, None, y_pred, loss)

    return Engine(_update)


def autoencoder_evaluator(
        model, metrics={}, 
        device=None, non_blocking=False, 
        prepare_batch=_prepare_batch, 
        output_transform=lambda x, y, y_pred: (y_pred, x,)):

    if device:
        model.to(device)

    def _inference(engine, batch):
        model.eval()
        with torch.no_grad():
            x, _ = prepare_batch(batch, device=device, non_blocking=non_blocking)
            y_pred = model(x)
            return output_transform(x, None, y_pred)

    engine = Engine(_inference)

    for name, metric in metrics.items():
        metric.attach(engine, name)

    return engine


def train(net, train_batch_size, val_batch_size, epochs, lr, 
        momentum, log_interval, log_dir):
    
    model = net
    device = 'cuda' if cuda.is_available() else 'cpu'
    train_loader, val_loader = get_data_loaders(
        train_batch_size, val_batch_size)
    writer = create_summary_writer(model, train_loader, log_dir)

    optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
    loss = nn.MSELoss()
    metrics = {'acc': Accuracy(), 'loss': Loss(loss)}

    trainer = autoencoder_trainer(model, optimizer, loss, device=device)
    evaluator = autoencoder_evaluator(model, metrics=metrics, device=device)
    
    #pbar = ProgressBar()
    #pbar.attach(trainer, output_transform=lambda x: {'loss': round(x,3)})

    @trainer.on(Events.ITERATION_COMPLETED)
    def log_training_loss(engine):
        print(
            'Epoch[{}] Loss: {:.4.f}'.format(
                engine.state.epoch, engine.state.loss
            )
        )
    
    @trainer.on(Events.ITERATION_COMPLETED)    
    def log_training_loss(engine):
        iter = (engine.state.iteration - 1) % len(train_loader) + 1
        if iter % log_interval == 0:
            print(
                "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}".format(
                    engine.state.epoch, iter, len(train_loader), 
                    engine.state.output
                )
            )     

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_trainer_results(engine):
        evaluator.run(train_loader)
        metrics = evaluator.state.metrics
        print(
            'Training results - Epoch: {} Avg accuracy: {:.4f} Avg loss: {:.4f}'
            .format(engine.state.epoch, metrics['acc'], metrics['loss'])
        )
        
    @trainer.on(Events.EPOCH_COMPLETED)
    def print_loss(engine):
        epoch = engine.state.epoch
        loss = engine.state.output['loss']
        print (
            'Epoch {epoch}: train_loss = {loss}'.format(epoch=epoch, loss=loss)
        )        

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(trainer):
        evaluator.run(val_loader)
        metrics = evaluator.state.metrics
        print(
            'Training results - Epoch: {} Avg accuracy: {:.4f} Avg loss: {:.4f}'
            .format(trainer.state.epoch, metrics['acc'], metrics['nll'])
        )

    trainer.run(train_loader, max_epochs=epochs)

### Training

In [0]:
model = VGGAE()

train(model, 256, 1000, 50, 0.01, 0.5, 10, './tensorboard_logs')

Error occurs, No graph saved
Checking if it's onnx problem...
Your model cannot be exported by onnx, please report to onnx team
Failed to save model graph: 'GraphDef' object does not support indexing


RuntimeError: ignored